# **Data Collection**

## Objectives

* Fetch data from Kaggle and prepare it for further processes.

## Inputs

*   Kaggle JSON file - the authentication token. 

## Outputs

* Generate Dataset: inputs/datasets/equine_breeds.

## Additional Comments | Insights | Conclusions


* No additional comments.

---

## Import Packages

In [2]:
%pip install -r /workspaces/Animal_detection_camera/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 44.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 47.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 25.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 49.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 30.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 48.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.2/214.2 MB 40.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 33.6 MB/s eta 0:00:00
   ━━

# Change working directory

Access the current directory

In [3]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/Animal_detection_camera/jupyter_notebooks'

Set a new directory

In [11]:
os.chdir('/workspaces/Animal_detection_camera')
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [12]:
current_dir = os.getcwd()
current_dir

'/workspaces/Animal_detection_camera'

# Install Kaggle

In [13]:
%pip install kaggle==1.5.12


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Change the Kaggle configuration directory to the current working directory and set permissions for the Kaggle authentication JSON.

In [14]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Set the Kaggle Dataset and download it. Then, unzip image files and delete the zip file.

In [18]:
import os
import zipfile

KaggleDatasetPath = "anthonytherrien/image-classification-64-classes-animal"
ZipFolder = "inputs/animals"
DestinationFolder = "inputs/datasets/animals"

# Download the dataset zip
!kaggle datasets download -d {KaggleDatasetPath} -p {ZipFolder}
zip_path = os.path.join(ZipFolder, "image-classification-64-classes-animal.zip")

os.makedirs(DestinationFolder, exist_ok=True)

# Unzip only if destination is empty
if not os.listdir(DestinationFolder):
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(DestinationFolder)
    print(f"✅ Extracted dataset to: {DestinationFolder}")
    # delete zip after extraction
    os.remove(zip_path)
else:
    print(f"📂 Dataset already available at: {DestinationFolder}")


100%|█████████████████████████████████████▉| 5.71G/5.71G [02:54<00:00, 34.8MB/s]
100%|██████████████████████████████████████| 5.71G/5.71G [02:54<00:00, 35.1MB/s]
✅ Extracted dataset to: inputs/datasets/animals


---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
    # create here your folder
    # os.makedirs(name='')
except Exception as e:
    print(e)
